# Python Applied: Programming for the Stars

**Nicholas Hunt-Walker**

**Code Fellows - Code 401: Python**

**March 18, 2016**

# Outline
### Less App Dev, More Analysis
### A Brief Survey of Astrophysics
### Time Series Analysis
### Use case: Catching Stellar Flares
### Stellar Classification
### Use case: Finding RR Lyrae
### Mapping the Sky
### Use case: Thousands of Galaxies, Billions of Years
### Summary

# Less App Dev, More Analysis

Code rarely (if ever) client-facing. At most, may share with collaborators

Custom scripts, sometimes legacy code

Math/Stats-heavy; get from point A (read data) to point B (analysis results)

### Sometimes code becomes library. Useful:
<img src='https://camo.githubusercontent.com/791c5b926e46cd3ce6b21e6e954e252095dc81c2/687474703a2f2f617374726f70792e72656164746865646f63732e6f72672f656e2f737461626c652f5f696d616765732f617374726f70795f62616e6e65722e737667' width="400"/>

<img src='./astroML_logo.gif' width="300"/>

### Other often-used libraries:
| <img src='numpy_logo.png' alt='numpy' width='300'/> | <img src='scipy_logo.png' alt='scipy' width='300'/> |
| - | - |
| <img src='scikit-learn-logo.png' alt='sklearn' width='300'/> | <img src='pandas_logo.png' alt='pandas' width='300'/> |
| <img src='matplotlib_logo.png' alt='matplotlib' width='300'/> | <img src='emcee_logo.png' alt='emcee' width='300'/> |

# A Brief, Largely Incomplete Survey of Astrophysics
<img src='hud2014_1280.jpg' width='600' alt='hubble_udf'/>
<em style='float: right'>The Hubble Ultra-Deep Field, Hubble Space Telescope 2014</em>

# Planets
<img src='the_planets.jpg' alt='the planets' width='800' />
- Tend to orbit stars, but not always
- Collections of mass large enough to be rounded by their own gravity
- Range drastically in size; 4,800 km (Mercury) to 320,000 km (CT Chamaeleontis b); United States is 4,300 km across
- Smallest are rocky (Mercury, Venus, Mars), largest mostly gas (Jupiter, Saturn Uranus)
- **What's studied?**
    - **Finding new planets!**
    - **Masses and sizes of new planets!**
    - **Atmospheres of...new planets!**

# Stars
<img src='pleiades_banner.jpg' width='800' alt='the pleiades'/>
<em style='float: right'>The Pleiades Star Cluster</em>
- Ridiculously huge balls of hydrogen plasma
- Ridiculously hot sources of light
- Ridiculously long-lived, even for shortest-lived ones
- Literally hundreds of billions in our own Milky Way galaxy
- **What's studied?**
    - **Variability in brightness with time**
    - **Internal physics of stars**
    - **Distribution throughout the sky**
    - **Chemical enrichment throughout bodies**

# Galaxies
<img src='andromeda_banner.jpg' alt='The Andromeda Galaxy' width='700' style='margin: auto;'/>
<em style='float: right'>The Andromeda Galaxy</em>
- Gravity-bound collections of hundreds of billions of stars
- Also at least as many planets, plus black holes, white dwarfs, neutron stars, & occasional supernovae
- Tons of gas and dust between stars allowing for new stars
- Cannibalism!
- Come in spiral and elliptical varieties
- **What's studied?**
    - **Distribution on sky**
    - **Structure & shape**
    - **Ages and content**
    - **Evolution with time**

# Time Series Analysis

<img src='time_series_rrlyr.jpg' alt='time series head' width='800'/>
- Investigate variation in some characteristic with time
- Data from observations or simulations; always testing physics
- **Used for**: 
    - **finding new planets**
    - **using stars as distance indicators**
    - **classifying stars**
    - **predicting galaxy merger histories**

# Use case: Catching Stellar Flares
<img src='stellar_flare.gif' width='500' alt='stellar flare' style='margin:auto;'/>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

the_file = 'lc.ADLeo.dat' 
# observations every 60 seconds
the_data = np.loadtxt(the_file).T
print "This data consists of %i rows and %i columns" % (len(the_data[0]), len(the_data.T[0]))

This data consists of 8592 rows and 2 columns


In [2]:
# Visualize the data
fig = plt.figure(figsize=(16, 8))
fig.subplots_adjust(wspace=0, hspace=0, top=0.95, bottom=0.05, left=0.05, right=0.95)

plt.subplot(211)
plt.plot(the_data[0], the_data[1], color='k')
plt.xlim(3716.75, 3726.25)
plt.xlabel('Time (days)')
plt.ylabel('Brightness')
plt.minorticks_on()

plt.subplot(212)
plt.scatter(the_data[0], the_data[1], color='k', s=1)
plt.xlim(3716.75, 3726.25)
plt.ylim(22, 23.5)
plt.xlabel('Time (days)')
plt.ylabel('Brightness')
plt.minorticks_on()
plt.savefig("lightcurve_1.png")
# plt.show()
plt.close()

<img src="lightcurve_1.png" width="800"/>

### My data appears to not only be periodic, but to be slightly decreasing in brightness over time. If I want to recover any flares from this object, I have to model that behavior and subtract it out. 

In [3]:
from scipy.optimize import leastsq

In [4]:
# when fitting a function to data you need to start with a first guess, 
# then optimize that function from there
guesses = {"amplitude": np.std(the_data[1]),
            "freq": 2*np.pi/3.0, 
            "phase": 0.,
            "intercept": np.mean(the_data[1]),
            "slope": 1.0}

# use better-named variables and shift the time to start at zero
time = the_data[0] - the_data[0].min()
brightness = the_data[1]

# y = A * sin(f*t + phase) + bias
optimize_func = lambda params: params[0] * np.sin(params[1] * time + params[2]) \
                                + (params[3] * time + params[4]) - brightness
    
results = leastsq(optimize_func, [guesses["amplitude"], guesses["freq"], 
                                  guesses["phase"], guesses["slope"], 
                                  guesses["intercept"]]) 

amplitude, freq, phase, slope, intercept = results[0]

In [5]:
def variability_model(amp, freq, phase, slope, intercpt, time):
    """
    A function modeling the variablility of this star, with a sinusoid 
    for the star spot and a linear bias for any long-term decline or increase
    """
    return amp*np.sin(freq * time + phase) + (slope * time + intercpt)

In [6]:
# always visualize your results. Pure numbers can only say so much

fig = plt.figure(figsize=(16, 8))
fig.subplots_adjust(wspace=0, hspace=0, top=0.95, bottom=0.05, left=0.05, right=0.95)

plt.subplot(211)
plt.scatter(time, brightness, color='k', s=1)
plt.plot(time, variability_model(amplitude, freq, phase, slope, intercept, 
                                 time), linewidth=2, color="red")

plt.xlim(-0.5, 9.5)
plt.ylim(22, 23.5)
plt.ylabel('Brightness')
plt.minorticks_on()

plt.subplot(212)
the_model = variability_model(amplitude, freq, phase, slope, intercept, time)

residual = brightness - the_model
plt.scatter(time, residual, color='k', s=1)
plt.plot([-1, 10], [np.std(residual), np.std(residual)],
         linestyle="--", c='b')

plt.xlim(-0.5, 9.5)
plt.ylim(-0.5, 1.5)
plt.xlabel('Time (days)')
plt.ylabel('Brightness')
plt.minorticks_on()

plt.savefig("lightcurve_2.png")
# plt.show()
plt.close()

<img src='lightcurve_2.png' width="800"/>

In [7]:
def crude_flare_finder(time_data, flux_data, time_window_days = 1.0):
    """
    Returns indices of data where brightness is above 1 sigma 
    for that window of time
    """
    time_steps = np.arange(time_data.min(), time_data.max() \
                           + time_window_days, time_window_days)
    
    flare_indices = np.zeros(len(time_data), dtype=bool)
    
    for ii in range(len(time_steps)-1):
        this_window = (time_data > time_steps[ii]) & \
                        (time_data < time_steps[ii + 1])
            
        stdev = np.std(flux_data[this_window])
        flare_indices[this_window] = flux_data[this_window] > stdev
        
    return flare_indices

In [8]:
# and here are the resulting flares
fig = plt.figure(figsize=(16,4))
fig.subplots_adjust(wspace=0, hspace=0, top=0.95, bottom=0.05, left=0.05, right=0.95)

crude_flares = crude_flare_finder(time, residual)

for ii in np.arange(0, 10, 2.0):
    plt.fill_between([ii, ii+1], [-0.5, -0.5], [1.5, 1.5], color="k", alpha=0.1)
    
plt.scatter(time, residual, color='k', s=5, edgecolor="None")
plt.scatter(time[crude_flares], residual[crude_flares], color='r', s=5, edgecolor="None")
plt.xlim(-0.5, 9.5)
plt.ylim(-0.5, 1.5)
plt.xlabel('Time (days)')
plt.ylabel('Brightness')
plt.minorticks_on()

plt.savefig("lightcurve_withflares.png")
# plt.show()
plt.close()

<img src="lightcurve_withflares.png" width="800" />

# Object Classification

<img src='hr_diagram.jpg' width='600' alt='H-R Diagram' />

### Different object classes have different physcs and different meanings

- Stellar classes = different phases of evolution
- Stellar classes also = different temperatures
- Galaxy classes = different stars in the galaxy + different contents
- **Classification lets us focus attention**
    - Sky filled with hundreds of billions of objects
    - Computing power/man hours limited
    - Classify then delegate

# Use case: Classifying RR Lyrae
<img src="cepheid.gif" width="600" alt="Pulsating Star" />

In [9]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from astroML.classification import GMMBayes
from astroML.datasets import fetch_rrlyrae_combined
from astroML.utils import split_samples
from astroML.utils import completeness_contamination

In [10]:
# get data and split into training & testing sets
X, y = fetch_rrlyrae_combined()
X = X[:, [1, 0, 2, 3]]  # rearrange columns for better 1-color results

# GMM-bayes takes several minutes to run, and is order[N^2]
#  truncating the dataset can be useful for experimentation.
(X_train, X_test), (y_train, y_test) = split_samples(X, y, [0.75, 0.25],
                                                     random_state=0)
N_tot = len(y)
N_st = np.sum(y == 0)
N_rr = N_tot - N_st
N_train = len(y_train)
N_test = len(y_test)
N_plot = 5000 + N_rr

print "This set has %i objects, with %i dimensions" % X.shape

This set has 93141 objects, with 4 dimensions


In [11]:
# perform classfication
Ncolors = np.arange(1, X.shape[1] + 1)
Ncomp = [1, 3]

def compute_GMMbayes(Ncolors, Ncomp):
    classifiers = []
    predictions = []

    for ncm in Ncomp:
        classifiers.append([])
        predictions.append([])
        for nc in Ncolors:
            clf = GMMBayes(ncm, min_covar=1E-5, covariance_type='full')
            clf.fit(X_train[:, :nc], y_train)
            y_pred = clf.predict(X_test[:, :nc])

            classifiers[-1].append(clf)
            predictions[-1].append(y_pred)

    return classifiers, predictions

classifiers, predictions = compute_GMMbayes(Ncolors, Ncomp)
completeness, contamination = completeness_contamination(predictions, y_test)

In [12]:
# Compute the decision boundary
clf = classifiers[1][1]
xlim = (0.7, 1.35)
ylim = (-0.15, 0.4)

xgrid = np.linspace(xlim[0], xlim[1], 71)
ygrid = np.linspace(ylim[0], ylim[1], 81)
xx, yy = np.meshgrid(xgrid, ygrid)

Z = clf.predict_proba(np.c_[yy.ravel(), xx.ravel()])
Z = Z[:, 1].reshape(xx.shape)

In [13]:
# plot the results
fig = plt.figure(figsize=(10, 5))
fig.subplots_adjust(hspace=0.0, wspace=0.2)

# left plot: data and decision boundary
ax = fig.add_subplot(121)
im = ax.scatter(X[-N_plot:, 1], X[-N_plot:, 0], c=y[-N_plot:],
                s=4, lw=0, cmap=plt.cm.binary, zorder=2)
im.set_clim(-0.5, 1)

im = ax.imshow(Z, origin='lower', aspect='auto', cmap=plt.cm.binary, zorder=1,
               extent=xlim + ylim)
im.set_clim(0, 1.5)

ax.contour(xx, yy, Z, [0.5], colors='k')
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xlabel('attribute 1')
ax.set_ylabel('attribute 2')

# plot completeness vs Ncolors
ax = fig.add_subplot(222)
ax.plot(Ncolors, completeness[0], '^--k', ms=6, label='N=%i' % Ncomp[0])
ax.plot(Ncolors, completeness[1], 'o-k', ms=6, label='N=%i' % Ncomp[1])

ax.xaxis.set_major_locator(plt.MultipleLocator(1))
ax.yaxis.set_major_locator(plt.MultipleLocator(0.2))
ax.xaxis.set_major_formatter(plt.NullFormatter())

ax.set_ylabel('completeness')
ax.set_xlim(0.5, 4.5)
ax.set_ylim(-0.1, 1.1)
ax.grid(True)

# plot contamination vs Ncolors
ax = fig.add_subplot(224)
ax.plot(Ncolors, contamination[0], '^--k', ms=6, label='N=%i' % Ncomp[0])
ax.plot(Ncolors, contamination[1], 'o-k', ms=6, label='N=%i' % Ncomp[1])
ax.legend(loc='lower right', bbox_to_anchor=(1.0, 0.78))

ax.xaxis.set_major_locator(plt.MultipleLocator(1))
ax.yaxis.set_major_locator(plt.MultipleLocator(0.2))
ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%i'))

ax.set_xlabel('N attributes')
ax.set_ylabel('contamination')
ax.set_xlim(0.5, 4.5)
ax.set_ylim(-0.1, 1.1)
ax.grid(True)

plt.savefig("classification_example.png")
# plt.show()
plt.close()

<img src="classification_example.png" width="800"/>

# Mapping the Sky
<img src='skymap.jpg' width='1000'/>

## Astronomy is a Visual Science
- Maps let us see the world in more views than what our eyes show
- Maps of our stars show us the galaxy; where they are and their properties by position
- Maps of galaxies show us the Universe
- Maps of the Universe show us our origins
- **Our code makes mapping the Universe in space and physics possible**

# Use case: Thousands of Galaxies, Billions of Years

In [14]:
import pandas as pd
import matplotlib.pyplot as plt

from astropy.cosmology import Planck13 as cosmo
from astropy.coordinates import SkyCoord, Distance
from astropy import units as u

from astroML.stats import binned_statistic_2d

import locale
locale.setlocale(locale.LC_ALL, 'en_US')

'en_US'

In [15]:
galaxies = pd.read_csv("/Users/Nick/Documents/AGBstuff/new_work/contaminants/vagc_allwise_allmags.dat", sep="\t")
galaxies = galaxies[galaxies.CLASS != "STAR"]
print "This file has %i rows and %i columns" % galaxies.shape

This file has 291578 rows and 47 columns


In [16]:
# want to see the distribution of galaxies but have to transform
# coordinates
coords = SkyCoord(ra = galaxies["ra"] * u.degree, 
                  dec = galaxies["decl"] * u.degree, 
                  distance = cosmo.luminosity_distance(z=galaxies["Z"]) * u.Mpc)

galaxies["x"] = coords.cartesian.x
galaxies["y"] = coords.cartesian.y
galaxies["z"] = coords.cartesian.z

qso = galaxies.CLASS == "QSO"
agn = (galaxies.SUBCLASS == "AGN") | (galaxies.SUBCLASS == " BROADLINE")
sfgal = (galaxies.SUBCLASS == "STARFORMING") | (galaxies.SUBCLASS == " BROADLINE")
sbgal = (galaxies.SUBCLASS == "STARBURST") | (galaxies.SUBCLASS == " BROADLINE")

In [17]:
#coordinates transformed very simply! Now visualize!
fig = plt.figure(figsize=(14, 6))
fig.subplots_adjust(wspace=0, hspace=0, top=0.95, bottom=0.05, left=0.05, right=0.95)

xbounds = [-600, 0]
ybounds = [-300, 300]
ax = plt.subplot2grid((2, 4),(0, 0), colspan=2, rowspan=2)
ax.scatter(galaxies["x"], galaxies["y"], s=1, edgecolor="None", c='k', alpha=0.5)
ax.text(0.95, 0.95, "All Galaxies", horizontalalignment="right",
       verticalalignment="top", transform=ax.transAxes)

ax.set_xlabel("X (Mpc)")
ax.set_ylabel("Y (Mpc)")
ax.set_xlim(xbounds[0], xbounds[1])
ax.set_ylim(ybounds[0], ybounds[1])
ax.minorticks_on()

size = 1
ax1 = plt.subplot2grid((2, 4),(0, 2))
ax1.scatter(galaxies["x"][sfgal], galaxies["y"][sfgal], s=size, edgecolor="None", c='gray', alpha=0.5)
ax1.text(0.95, 0.95, "Starforming", horizontalalignment="right",
       verticalalignment="top", transform=ax1.transAxes)

ax2 = plt.subplot2grid((2, 4),(0, 3))
ax2.scatter(galaxies["x"][qso], galaxies["y"][qso], s=size, edgecolor="None", c='b')
ax2.text(0.95, 0.95, "QSOs", horizontalalignment="right",
       verticalalignment="top", transform=ax2.transAxes)

ax3 = plt.subplot2grid((2, 4),(1, 2))
ax3.text(0.95, 0.95, "Starburst", horizontalalignment="right",
       verticalalignment="top", transform=ax3.transAxes)
ax3.scatter(galaxies["x"][sbgal], galaxies["y"][sbgal], s=size, edgecolor="None", c='g', alpha=0.5)

ax4 = plt.subplot2grid((2, 4),(1, 3))
ax4.scatter(galaxies["x"][agn], galaxies["y"][agn], s=size, edgecolor="None", c='r')
ax4.text(0.95, 0.95, "AGN", horizontalalignment="right",
       verticalalignment="top", transform=ax4.transAxes)

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_xlim(xbounds[0], xbounds[1])
    ax.set_xticklabels([])
    ax.set_ylim(ybounds[0], ybounds[1])
    ax.set_yticklabels([])
    ax.minorticks_on()

plt.savefig("galaxymap.png")
# plt.show()
plt.close()

<img src="galaxymap.png" />

In [18]:
# more than just classes, let's map color distributions
fig = plt.figure(figsize=(14, 8))
fig.subplots_adjust(wspace=0, hspace=0, top=0.95, bottom=0.05, left=0.05, right=0.95)

w23bins = np.arange(1, 4.5, 0.5)
xbounds = [-600, 0]
ybounds = [-300, 300]
boxprops = {"facecolor":'white', 'edgecolor':'None'}

for ii in range(6):
    ax = plt.subplot(2, 3, ii + 1)
    cut = ((galaxies["w2"] - galaxies["w3"]) > w23bins[ii]) &\
            ((galaxies["w2"] - galaxies["w3"]) < w23bins[ii + 1])
    ax.scatter(galaxies["x"][cut], galaxies["y"][cut], s=0.5, edgecolor="None", c='k', alpha=0.9)
    
    ax.text(0.95, 0.95, "%g > color > %g" % (w23bins[ii], w23bins[ii+1]), horizontalalignment="right",
           verticalalignment="top", transform=ax.transAxes,
           bbox = boxprops)
    ax.text(0.95, 0.85, "%s galaxies" % locale.format("%g", sum(cut), grouping=True), horizontalalignment="right",
           verticalalignment="top", transform=ax.transAxes,
           bbox = boxprops)

    ax.set_xlim(xbounds[0], xbounds[1])
    ax.set_xticklabels([])
    ax.set_ylim(ybounds[0], ybounds[1])
    ax.set_yticklabels([])
    ax.minorticks_on()

plt.savefig("galaxymap_color.png")
# plt.show()
plt.close()

<img src="galaxymap_color.png" />

# And so, so much more

### Not just direct science!
- Astronomers use Python to help aim telescopes
- Example: The Large Synoptic Survey Telescope
<img src='lsst.jpg' />

### Not just observations!
- Python backends on sites like:
    - [SPACEPROB.ES](http://spaceprob.es/)
    - [Open Exoplanet Catalog](http://www.openexoplanetcatalogue.com/systems/)

### Also massive data sumaries!

In [21]:
from datetime import timedelta

class YouTubeVideo(object):
    def __init__(self, id, width=400, height=300, start=timedelta()):
        self.id = id
        self.width = width
        self.height = height
        self.start = start.total_seconds()

    def _repr_html_(self):
        return """
                <iframe width="%i" height="%i"
                    src="http://www.youtube.com/embed/%s?start=%i"
                    frameborder="0" allowfullscreen style='margin: auto;'>
                </iframe>
               """ % (self.width, self.height, self.id, self.start)

In [22]:
YouTubeVideo("_DnDeBa0KFc", width=800, height=450, start=timedelta(hours=0, minutes=0, seconds=0))

# Questions?